<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from datetime import timedelta, datetime, date

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
VERSION = 6

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
VAL_SIZE   = 0.20   # 20%

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Data

In [ ]:
!ls '../input'

In [ ]:
data_dir = '../input/parsing-all-moscow-auto-ru-09-09-2020/'

In [ ]:
# сентябрьский датасет, оставляю для сравнения цен, т.к. со свежими данными щансов попасть в начало лидерборда НЕТ
train_last = pd.read_csv('../input/parsing-all-moscow-auto-ru-09-09-2020/all_auto_ru_09_09_2020.csv') 

# тестовый датасет
test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')   

# спарсенный мною датасет для обучения модели
train = pd.read_csv('../input/all-cars-sitecsv/all_cars_site.csv', sep=';')
train = train.drop(['N'], axis=1)

# submission
sample_submission = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

In [ ]:
test.head()

In [ ]:
train.head()

### Посмотрим на пропуски в тренировочном и тестовом датасетах:

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
sns_heatmap = sns.heatmap(train.isnull(), yticklabels=False, cbar=False, cmap='viridis')

##### Видим, что в спарсеном датасете пропускосв нет

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
sns_heatmap = sns.heatmap(test.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
train.info()

In [ ]:
test.info()

### Из приведенных выше данных видно, что для обучения модели можно использовать только следующие столбцы:

In [ ]:
columns = ['model_name',
           'bodyType', 
           'brand', 
           'productionDate', 
           'engineDisplacement', 
           'mileage',  
           'enginePower',  
           'fuelType', 
           'Привод', 
           'Руль', 
           'vehicleTransmission']

#### Из них числовые:

In [ ]:
num_cols = ['productionDate', 
            'engineDisplacement',
            'enginePower',
            'mileage']  

#### Из них категориальные

In [ ]:
cat_cols = ['model_name',
            'bodyType', 
            'brand',  
            'fuelType', 
            'Привод', 
            'Руль', 
            'vehicleTransmission']

# Приведение train и test к одинаковому виду

## brand

In [ ]:
train['brand'].value_counts()

In [ ]:
train['brand'] = train['brand'].apply(lambda x : x.upper())


def get_mersedes(brand):
    """Приведение названия бренда  одинаковому виду"""
    if brand == 'MERCEDES-BENZ':
        return 'MERCEDES'
    else:
        return brand
 

train['brand'] = train['brand'].apply(get_mersedes)

In [ ]:
train['brand'].value_counts()

## model_name

In [ ]:
train['model_name'].value_counts()

In [ ]:
train['model_name'].value_counts().index

In [ ]:
def del_space(x):
    """Удаление лишних пробелов"""
    return x.strip()

In [ ]:
train['model_name'] = train['model_name'].apply(del_space)

In [ ]:
train['model_name'].value_counts().index

### Удаление неиспользуемых в прогнозе значений model_name

In [ ]:
train['model_name'].value_counts()

In [ ]:
test_names = test.model_name.value_counts().index
train_names = train.model_name.value_counts().index

model_names = []  # перечень моделей, отсутствующих в тестовом датасете
for name in train_names:
    if name not in test_names:
        model_names.append(name)

In [ ]:
model_names

In [ ]:
def highlight_mod_names(name):
    """Выделение отсутствующих моделей"""
    if name in model_names:
        return 'отсутствует'
    else:
        return name

In [ ]:
train['model_name'] = train['model_name'].apply(highlight_mod_names)

In [ ]:
def describe_df_collumn(data_frame, collumn):
    """Функция, использовавшая при анализе и обработке спарсенных данных"""
    print('Группировка по значениям:' )
    print(data_frame[collumn].value_counts().head(7))
    print()
    print('Нулевые значения' )
    print(data_frame[collumn].isna().value_counts())

    ind = data_frame[data_frame[collumn].str.contains('отсутствуе', case=False) ].index
    print()
    if len(ind) != 0:
        data_frame.drop(ind, inplace=True)
        print( 'Удалено строк со значением "отсутствует"', len(ind) )
    else: 
        print( 'Строк со значением "отсутствует" нет')    


In [ ]:
describe_df_collumn(train, 'model_name')

### Price

# Демонстрация изменения ценового диапазона за последние 6 месяцев 

#### Удаление неиспользуемых юрендов из приложенного в ЛБ тренировочного датасета

In [ ]:
ind = train_last[ (train_last['brand'] != 'AUDI') &
                  (train_last['brand'] != 'BMW') &
                  (train_last['brand'] != 'HONDA') &
                  (train_last['brand'] != 'INFINITI') &
                  (train_last['brand'] != 'LEXUS') &
                  (train_last['brand'] != 'MERCEDES') &
                  (train_last['brand'] != 'MITSUBISHI') &
                  (train_last['brand'] != 'NISSAN') &
                  (train_last['brand'] != 'SKODA') &
                  (train_last['brand'] != 'TOYOTA') &
                  (train_last['brand'] != 'VOLKSWAGEN') &
                  (train_last['brand'] != 'VOLVO') ].index
print(len(ind))
train_last = train_last.drop(ind)

In [ ]:
train_last_price = train_last.groupby(by=["brand"]).price.mean()
train_price = train.groupby(by=["brand"]).price.mean()

Вычислим изменение цен за последние полгода как отношение:  Старые цены / Новые цены

In [ ]:
price_difference = {}
for brand in train_last_price.index:
    price_difference[brand] = round(train_last_price[brand]/train_price[brand],3)
price_difference

видно, что некоторые бренды выросли в цене, в то время как другие, наоборот, подешевели

# Числовые признаки

In [ ]:
for col in num_cols:
    fig = plt.figure(figsize=(9, 5))
    gs = fig.add_gridspec(1, 2)
        
    ax = fig.add_subplot(gs[0, 0])
    sns.distplot(train[col], kde = False, rug=False)

    ax = fig.add_subplot(gs[0, 1])
    sns.boxplot(data=train[col])

    fig.tight_layout()

### Значимость числовых признаков

In [ ]:
imp_num = pd.Series(f_classif(train[num_cols], train['price'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

### Корреляция

In [ ]:
sns.heatmap(train[num_cols].corr().abs(), annot = True, vmin=0, vmax=0.5)

# Категориальные признаки

In [ ]:
df = train[columns]
df['price'] = train['price']

In [ ]:
for column in cat_cols:
    df[column] = df[column].astype('category').cat.codes

## Визуализация категориальных переменных

In [ ]:
for col in cat_cols:

    fig = plt.figure(figsize=(10, 4))
    gsp = fig.add_gridspec(1, 2)

    ax = fig.add_subplot(gsp[0, 0])
    sns.boxplot(y=df[col])

    ax = fig.add_subplot(gsp[0, 1])
    sns.barplot(x=col, y='price', data=df, estimator=len)

    fig.tight_layout()

## Значимость категориальных переменных

In [ ]:
for column in cat_cols:
    df[column] = df[column].astype('category').cat.codes

In [ ]:
imp_cat = pd.Series(mutual_info_classif(df[cat_cols], 
                                        df['price'],
                                        discrete_features =True), 
                    index = (cat_cols))

imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

### Когда все переменные категориальные

In [ ]:
imp_cat = pd.Series(mutual_info_classif(df[columns], df['price'],
                                     discrete_features =True), index = columns)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

## Data Preprocessing

In [ ]:
# train.dropna(subset=['productionDate','mileage'], inplace=True)
train.dropna(subset=['price'], inplace=True)

In [ ]:
df_train = train[columns] 
df_test  = test[columns]
# df_train = train[['mileage', 'model_name', 'enginePower']]
# df_test =   test[['mileage', 'model_name', 'enginePower']]

In [ ]:
y = train['price']

In [ ]:
y

## Label Encoding

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data

In [ ]:
for column in columns:
    data[column] = data[column].astype('category').cat.codes

In [ ]:
data

In [ ]:
X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам mileage в трейне и размечаем тест
predict = X_test['mileage'].map(tmp_train.groupby('mileage')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 18%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
np.log(y_train)

In [ ]:
random_grid = {'iterations': [1000, 2000, 3000, 4000, 5000, 6000],
               'custom_metric': ['R2', 'MAE'],
               'eval_metric': ['R2', 'MAE'],
               'silent': [True, False]}

model = CatBoostRegressor(random_seed = RANDOM_SEED,
                          eval_metric='MAPE')

model_random = RandomizedSearchCV( estimator = model, 
                                   param_distributions = random_grid, 
                                   n_iter = 1, 
                                   cv = 3, 
                                   verbose=10, 
                                   random_state=RANDOM_SEED, 
                                   n_jobs = -1)

model_random.fit(X_train, np.log(y_train))

In [ ]:
print('model_random.best_params_: ', model_random.best_params_)

In [ ]:
model = CatBoostRegressor(iterations = 6000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric='R2',
                          silent=True)

model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
#        use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))
predict_submission

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

# Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing

## Random Forest

In [ ]:
start = datetime.now()

random_grid = {'n_estimators': [100, 200, 300, 400],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [3, 5, 7, 9, 11, 13, 15],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

rfr = RandomForestRegressor(random_state = RANDOM_SEED)

rf_random = RandomizedSearchCV(estimator = rfr, 
                               param_distributions = random_grid, 
                               n_iter = 10, 
                               cv = 3, 
                               verbose=10, 
                               random_state=RANDOM_SEED, 
                               n_jobs = -1)

rf_random.fit(X_train, np.log(y_train))


best_rfr = RandomForestRegressor( random_state=RANDOM_SEED,
                                  n_estimators=400,
                                  min_samples_split=2,
                                  min_samples_leaf=1,
                                  max_features='auto',
                                  max_depth=15,
                                  bootstrap=True)

best_rfr.fit(X_train, np.log(y_train))


predict_rfr = np.exp(best_rfr.predict(X_test))

print('rf_random.best_params_: ', rf_random.best_params_)
finish = datetime.now()
print('time', finish-start)
print(f"Точность по метрике MAPE:{(mape(y_test, predict_rfr))*100:0.2f}%")

на большем количестве итераций значения другие, не стал тратить время...

## Gradient Boosting Regressor

In [ ]:
start = datetime.now()

random_grid = {'n_estimators': [100, 160, 220, 280, 340, 400],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth': [3, 5, 7, 9, 11, 13, 15],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4]}

gbr = GradientBoostingRegressor()

gbr_random = RandomizedSearchCV(estimator = gbr, 
                                param_distributions = random_grid, 
                                n_iter = 10, 
                                cv = 3, 
                                verbose=10, 
                                random_state=RANDOM_SEED, 
                                n_jobs = -1)

gbr_random.fit(X_train, np.log(y_train))

best_gbr = GradientBoostingRegressor( random_state=RANDOM_SEED,
                                      n_estimators=800,
                                      min_samples_split=5,
                                      min_samples_leaf=4,
                                      max_features='sqrt',
                                      max_depth=9)

best_gbr.fit(X_train, np.log(y_train))

predict_gbr = np.exp(best_gbr.predict(X_test))

print(gbr_random.best_params_)
finish = datetime.now()
print('time', finish-start)
print(f"Точность по метрике MAPE:{(mape(y_test, predict_gbr))*100:0.2f}%")

# Бэггинг 

Со случайным лесом

In [ ]:
start = datetime.now()

bagg_rfr = BaggingRegressor(best_rfr, 
                            n_estimators=3, 
                            n_jobs=1, 
                            random_state=RANDOM_SEED)

bagg_rfr.fit(X_train, np.log(y_train))

predict_bagg_rfr = np.exp(bagg_rfr.predict(X_test))

finish = datetime.now()
print('time', finish-start)
print(f"Точность по метрике MAPE:{(mape(y_test, predict_bagg_rfr))*100:0.2f}%")

С градиентным бустингом

In [ ]:
start = datetime.now()

bagg_gbr = BaggingRegressor(best_gbr, 
                            n_estimators=3, 
                            n_jobs=1, 
                            random_state=RANDOM_SEED)

bagg_gbr.fit(X_train, np.log(y_train))

predict_bagg_gbr = np.exp(bagg_gbr.predict(X_test))

finish = datetime.now()
print('time', finish-start)
print(f"Точность по метрике MAPE:{(mape(y_test, predict_bagg_gbr))*100:0.2f}%")

XGB Regressor

In [ ]:
start = datetime.now()

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', 
                           colsample_bytree=0.5,
                           learning_rate=0.05, 
                           max_depth=10, 
                           alpha=1,
                           n_estimators=1000)

xgb_reg.fit(X_train, np.log(y_train))

xgb_red_pred = np.exp(xgb_reg.predict(X_test))

finish = datetime.now()
print('time', finish-start)
print(f"Точность по метрике MAPE:{(mape(y_test, xgb_red_pred))*100:0.2f}%")

# Стеккинг

In [ ]:
start = datetime.now()

estimators=[('b_gbr', 
             BaggingRegressor(GradientBoostingRegressor(random_state=RANDOM_SEED,
                                                        n_estimators=800,
                                                        min_samples_split=5,
                                                        min_samples_leaf=4,
                                                        max_features='sqrt',
                                                        max_depth=9),
             n_estimators=3,
             n_jobs=1,
             random_state=RANDOM_SEED)),
            ('xgb', xgb.XGBRegressor(objective='reg:squarederror',
                                     colsample_bytree=0.5,
                                     learning_rate=0.05,
                                     max_depth=12,
                                     alpha=1,
                                     n_estimators=1000))]

st_ensemble = StackingRegressor(estimators=estimators,
                                final_estimator = CatBoostRegressor(iterations = 5000,
                                                                    random_seed = RANDOM_SEED,
                                                                    eval_metric='MAPE',
                                                                    custom_metric=['R2', 'MAE'],
                                                                    silent=True,
                                                                    learning_rate=0.13,
                                                                    depth=12,
                                                                    l2_leaf_reg=8,
                                                                    random_strength=0.3))

    
st_ensemble.fit(X_train, np.log(y_train))

predict_ensemble = np.exp(st_ensemble.predict(X_test))

finish = datetime.now()
print('time', finish-start)
print(f"Точность по метрике MAPE:{(mape(y_test, predict_ensemble))*100:0.2f}%")